In [1]:
%matplotlib inline

#plt.figure(figsize=(30,30))
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson

In [2]:
# Jack's car rental policy iteration


In [3]:
# step 2 Policy Evaluation
def PR(c1, c2):
    pssa = 0.0
    rssa = 0.0
    if c1 > c2:
        for i in range(c2+1):
            pi = poisson.pmf(c1-c2+i, dec_mu1) * poisson.pmf(i, inc_mu1)
            pssa += pi
            rssa += pi * (sale_reward*(c1-c2+i))
    else:
        for i in range(c1+1):
            pi = poisson.pmf(i, dec_mu1) * poisson.pmf(c2-c1+i, inc_mu1)
            pssa += pi
            rssa += pi *(sale_reward*i)
    return pssa, rssa

def Q(s, a, s_next):
    s_start = (s[0] - a, s[1] + a)
    c1, c2 = s_start
    c1n, c2n = s_next
    pssa = 0.0
    rssa = 0.0
    #print c1, c2, c1n, c2n, value_s[s_next]
    if (c1, c1n, a, 0) in Qcache:
        pssa, rssa = Qcache[(c1, c1n, a, 0)]
    else:
        pssa, rssa = PR(c1, c1n)
        Qcache[(c1, c1n, a, 0)] = (pssa, rssa)
    
    if (c2, c2n, a, 1) in Qcache:
        p, r= Qcache[(c2, c2n, a, 1)]
        pssa *= p
        rssa += r
    else:
        p, r = PR(c2, c2n)
        pssa *= p
        rssa += r
        Qcache[(c2, c2n, a, 1)] = (pssa, rssa)
    ret = pssa *(rssa - drive_cost * abs(a) + discount * value_s[s_next])
    return ret

In [4]:
def Qa(s, a):
    qa = 0
    for s_next in value_s:
        qa += Q(s, a, s_next)
    return qa

In [5]:
def update(s):
    acts = actions[s]
    v = 0
    Pa = 1.0/len(acts) # equiprobable
    for a in acts:
        v += Pa * Qa(s, a)
    return v

max_cars = 10
max_change = 5
sale_reward = 10
drive_cost = 2
dec_mu1 = 3
inc_mu1 = 3
dec_mu2 = 4
inc_mu2 = 2
value_s = {}
discount = 0.9
actions = {}
for i in range(max_cars):
    for j in range(max_cars):
        value_s[(i,j)] = 0.09
        # actions is to move from location 1 to location 2. negative means from 2 to 1.
        actions[(i,j)] = [x for x in range(i, -j-1, -1) if x<=5 and x >=-5
                          and i-x <= max_cars and -j-x >= -max_cars]
Qcache = {}
theta = 0.1
delta = 1
count = 0
while (delta > theta):
    delta = 0
    count += 1
    for s in value_s:
        tmp = value_s[s]
        value_s[s] = update(s)
        if delta < abs(tmp - value_s[s]):
            delta = abs(tmp - value_s[s])
            max_s = s
    print "count=", count, " delta=", delta, " max_s=", max_s
        

count= 1  delta= 0.32709769569  max_s= (6, 9)
count= 2  delta= 0.141377392564  max_s= (6, 9)
count= 3  delta= 0.00040925795084  max_s= (6, 9)
